In [27]:
import sys
import re
import urllib3
import ssl
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from cStringIO import StringIO

In [28]:
def pdfparser(data):
    fp = file(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()

    return data

In [29]:
def pdffinder(patentnumber, trialnumber):
    webaddress = "https://ptabdata.uspto.gov/ptab-api/trials/"
    url = webaddress+trialnumber
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    webdata = r.data
    check = webdata.find('FWD Entered')
    if check == -1:
        return None, 0
    start_link = webdata.find('documents')
    start_quote = webdata.find('"https:', start_link)
    end_quote = webdata.find('"', start_quote + 1)
    link = webdata[start_quote + 1:end_quote]
    
    s = http.request('GET', link)
    value = s.data
    start_pdf = value.find('"type" : "final decision"')
    rel_download = value.find('download', start_pdf)
    start_pdflink = value.find('"https:', rel_download)
    end_pdflink = value.find('"', start_pdflink + 1)
    pdflink = value[start_pdflink + 1:end_pdflink]
    return pdflink

In [30]:
def datamine(web):
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    f = urllib.urlopen(web, context=ctx)
    with open("temp.pdf", "wb") as code:
        code.write(f.read())

    text = pdfparser("temp.pdf")
    searchObj = re.compile( r'\d,\d\d\d,\d\d\d',re.IGNORECASE)
    numbers = searchObj.findall(text)
    numbers = list(set(numbers))
    similar_numbers = [x for x in numbers if x != patentnumber]  

    return similar_numbers

In [31]:
patentnumber = '7,860,222'
trialnumber = "IPR2014-01278"
x = pdffinder(patentnumber,trialnumber)
y = datamine(x)

In [32]:
print y

['7,191,133', '6,141,406', '6,064,963', '6,668,045']
